<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/hw11_btcs_stability_head_1d_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, import modules and define parameters as supplied.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sl

In [0]:
'''=======
parameters
======='''

al = 1
dx = .2 #space mesh size
rho = 1. #conditinal number, to be less than .5 for the stability
dt = rho*(dx**2)/al #time step size

a = 0
b = 1
f = lambda x: np.sin(np.pi * x)
es = lambda x, t: np.exp(-1*(np.pi**2)*t) * np.sin(np.pi*x)
N = (b - a) / dx #Should be 5, given parameters.


Now, we need to use both FTCS and BTCS to solve this equation and find the supnorm of the error function.

For FTCS, we can utilize the same FTCS() code we used last week; for BTCS, a new code must be developed.

In [0]:
def heat_ftcs_err(dx,rho,al,exact_sol,phi,a,b,n):
  dt = rho*(dx**2)/al
  
  #Initialize with n = 0.
  j_0_linspace = np.linspace((a-n),(b+n),(b+n)-(a-n)+1)
  u_j_0 = np.zeros(len(j_0_linspace))
  for i in range(len(j_0_linspace)):
    u_j_0[i] = phi(j_0_linspace[i]*dx)
  
  #Initialize True Values Matrix.
  true_val = np.zeros(len(j_0_linspace))
  for j in range(len(j_0_linspace)):
    true_val[j] = exact_sol(j_0_linspace[j]*dx,0)
  
  #Initialize error vector, apply max error value for n = 0.
  max_error_vec = np.zeros(n+1)
  err_j_0 = (u_j_0 - true_val)
  max_error_vec[0] = max(abs(err_j_0))
  
  #Initialize approximation for n = 1, to use as first "u_j_(k-1)"
  j_1_linspace = np.linspace((a-(n-1)),(b+(n-1)),(b+(n-1))-(a-(n-1))+1)
  u_j_1 = np.zeros(len(j_1_linspace))
  for i in range(len(u_j_1)):
    j = i + 1
    u_j_1[i] = rho*u_j_0[j-1] + (1-2*rho)*u_j_0[j] + rho*u_j_0[j+1]
  
  u_j_k_minus_1 = u_j_1
  former_linspace = j_1_linspace
  
  #Find n = 1 max error for vector.
  true_val = np.zeros(len(j_1_linspace))
  for j in range(len(j_1_linspace)):
    true_val[j] = exact_sol(j_1_linspace[j]*dx,dt)
  err_j_1 = (u_j_1 - true_val)
  max_error_vec[1] = max(abs(err_j_1))
  
  #Perform iterative loop for all n >= 2.
  for k in range(2,n+1):
    #Create new linspace for u_j_k.
    new_linspace = np.linspace(a-(n-k),b+(n-k),(b+(n-k))-(a-(n-k))+1)
    u_j_k = np.zeros(len(new_linspace))
    
    #Iterate and fill u_j_k.
    for i in range(len(new_linspace)):
      j = i + 1
      u_j_k[i] = rho*u_j_k_minus_1[j-1] + (1 - 2*rho)*u_j_k_minus_1[j] + rho*u_j_k_minus_1[j+1]
    
    #Produce true values for comparison, and append to max vector.
    true_val = np.zeros(len(new_linspace))
    for i in range(len(new_linspace)):
      true_val[i] = exact_sol(new_linspace[i]*dx,k*dt)
    err_j_k = (u_j_k - true_val)
    max_error_vec[k] = max(abs(err_j_k))
    
    #Identify new values for former_linspace and u_j_(k-1), which allows us to
    #Continue the loop.
    former_linspace = new_linspace
    u_j_k_minus_1 = u_j_k
  
  true_err_max = max(max_error_vec)
  for i in range(len(max_error_vec)):
    if true_err_max == max_error_vec[i]:
      err_n = i
      
  return true_err_max, err_n, max_error_vec

In [5]:
output_1 = heat_ftcs_err(dx, rho, al, es, f, a, b, 5)
print("Supnorm of error for FTCS is " + str(output_1[0]))

Supnorm of error for FTCS is 0.06854713922733136


In [0]:
def heat_btcs_err(dx, rho, alpha, exact_sol, phi, a, b, n):
  dt = rho*(dx**2)/al
  rho_vec = [-1*rho, (1 + 2*rho), -1*rho]
  
  #Initialize with n = 0.
  j_0_linspace = np.linspace((a-n),(b+n),(b+n)-(a-n)+1)
  u_j_0 = np.zeros(len(j_0_linspace))
  for i in range(len(j_0_linspace)):
    u_j_0[i] = phi(j_0_linspace[i]*dx)
  
  #Initialize True Values Matrix.
  true_val = np.zeros(len(j_0_linspace))
  for j in range(len(j_0_linspace)):
    true_val[j] = exact_sol(j_0_linspace[j]*dx,0)
  
  #Initialize error vector, apply max error value for n = 0.
  max_error_vec = np.zeros(n+1)
  err_j_0 = (u_j_0 - true_val)
  max_error_vec[0] = max(abs(err_j_0))
  
  #Initialize approximation for n = 1, to use as first "u_j_(k-1)"
  j_1_linspace = np.linspace((a-(n-1)),(b+(n-1)),(b+(n-1))-(a-(n-1))+1)
  rho_matrix = np.zeros((len(j_1_linspace),len(j_1_linspace)))
  for i in range(len(j_1_linspace)-2):
    j = i + 1
    rho_matrix[i,j-1] = -1*rho
    rho_matrix[i,j] = 1 + 2*rho
    rho_matrix[i,j+1] = 1*rho
  rho_matrix[0,:] = 0
  rho_matrix[len(j_1_linspace)-1,:] = 0
  u_j_1 = sl.solve(rho_matrix,u_j_0)
  #Ax = b where
  #A := rho_matrix
  #x := u_j_k
  #b := u_j_k_minus_1
  
  u_j_k_minus_1 = u_j_1
  former_linspace = j_1_linspace
  
  #Find n = 1 max error for vector.
  true_val = np.zeros(len(j_1_linspace))
  for j in range(len(j_1_linspace)):
    true_val[j] = exact_sol(j_1_linspace[j]*dx,dt)
  err_j_1 = (u_j_1 - true_val)
  max_error_vec[1] = max(abs(err_j_1))
  
  #Perform iterative loop for all n >= 2.
  for k in range(2,n+1):
    #Create new linspace for u_j_k.
    new_linspace = np.linspace(a-(n-k),b+(n-k),(b+(n-k))-(a-(n-k))+1)
    
    #Fill u_j_k by scipy.linalg.solve().
    new_rho_matrix = np.zeros((len(new_linspace),len(new_linspace)))
    for i in range(len(new_linspace)-2):
      j = i + 1
      new_rho_matrix[i+1,j-1] = -1*rho
      new_rho_matrix[i+1,j] = 1 + 2*rho
      new_rho_matrix[i+1,j+1] = 1*rho
    new_rho_matrix[0,:] = 0
    new_rho_matrix[len(new_linspace)-1,:] = 0
    u_j_k = sl.solve(new_rho_matrix,u_j_k_minus_1)
    
    
    #Produce true values for comparison, and append to max vector.
    true_val = np.zeros(len(new_linspace))
    for i in range(len(new_linspace)):
      true_val[i] = exact_sol(new_linspace[i]*dx,k*dt)
    err_j_k = (u_j_k - true_val)
    max_error_vec[k] = max(abs(err_j_k))
    
    #Identify new values for former_linspace and u_j_(k-1), which allows us to
    #Continue the loop.
    former_linspace = new_linspace
    u_j_k_minus_1 = u_j_k
  
  true_err_max = max(max_error_vec)
  for i in range(len(max_error_vec)):
    if true_err_max == max_error_vec[i]:
      err_n = i
      
  return true_err_max, err_n, max_error_vec

In [25]:
output_2 = heat_btcs_err(dx,rho,al,es,f,a,b,5)
print("Supnorm of error for BTCS is " + str(output_2[0]))

ValueError: ignored

In [17]:
x = np.ones((5,5))
x[0,:] = 0
print(x)

[[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
